# Ethical Moderation

Note: The documentation for this lab can be found [here](https://dylanirlbeck.github.io/ethical-moderation). Various sections will be linked throughout this lab.

## Background and Learning Objectives

Read the [background](https://dylanirlbeck.github.io/ethical-moderation/project#background) and [learning objectives](https://dylanirlbeck.github.io/ethical-moderation/project#learning-objectives).


***

## Lab
_Sport-It_ is a popular social media platform known for its unique, sports-based community. Over the last few months,
_Sport-It_ has been rapidly growing in scale: the number of daily active users increased by 200% in the last month alone!
As a result, there has been an explosion in the number of posts made on the platform. It is untenable for the moderators
at _Sport-It_ to continue conducting manual review of posts, and so they’ve hired you to design an automated content moderation system.

These moderators are very intentional about their moderation: they only allow posts about sports. Topics like politics, finance,
and cooking, for example, are not allowed on _Sport-It_. They’ve directed you to write software that removes all posts not directly
related to sports.


## Required Libraries
Required libraries:
    * beautifultable

In [11]:
%load_ext autoreload
%autoreload 2

### Part 1: "Naive" Moderation

See https://dylanirlbeck.github.io/ethical-moderation/project#part-1-naive-moderation.

In [12]:
from lab import __version__, data_tools, display, utilities
from math import exp, log

print('Content filtering lab, version', __version__)

Content filtering lab, version 0.1


### Using the Training Data
We must use training data to “train” Naive Bayes. This training data consists of real posts from Sport-It pre-labeled as "on-topic" or "off-topic" by the moderators.

Each datapoint in the training set contains __text (the post title)__, and a label __'y' or 'n' (whether the post is on-topic or off-topic, respectively)__. Here is an example:
<blockquote>y: Musgrove throws first no-hitter in Padres history.</blockquote>
Here, the moderators have labeled this post as on-topic, as it fully pertains to baseball.

In [35]:
# If you'd like more training data, you can increase the limit value.
#
training_data = data_tools.parse_data('./data/politics.csv', './data/espn.txt', limit=300)

Now that we parsed our data, let's take a look at some of the data points to get a sense of how the moderators think.

As you look through the dataset, ask yourself (alternatively, if you're working in a group, have a discussion): __Do you agree with the labels provided? What would you change?__

In [36]:
display.display_labelled_data(training_data)

+--------+--------------------------------------------------------------------+
| Valid? |                      Title of post submission                      |
+--------+--------------------------------------------------------------------+
|   y    |      Turner homers, Dodgers beat Nats 1-0 on champs' ring day      |
|        |                                                                    |
+--------+--------------------------------------------------------------------+
|   n    | DeVos undoes Obama student loan protections; defaults expected to  |
|        |                                rise                                |
+--------+--------------------------------------------------------------------+
|   n    |       Trumps Reversals Come After Crash Course on the Issues       |
+--------+--------------------------------------------------------------------+
|   n    |     EPA: US Steel leaks chemical into Lake Michigan tributary      |
+--------+------------------------------

### Evaluate the training data

There are some posts, however, which still need labelling for the training set; the moderators have tasked you with assigning such labels. Each of these posts is not clearly on-topic or off-topic, so the moderators want you to decide.


Before you go to implement the Naive Bayes algorithm, the moderators email you some additional data that they want to include in the training set. These posts apparently sparked significant internal discussion, since, in the eyes of the moderators, they aren't clearly on-topic or off-topic. 

As the lead engineer, you have considerable say over the design of the algorithm and curation of the training set. Before you "extend" the training data with these new sample posts, look through the labels assigned to the posts. Do you fully agree with the labels?

If you change any labels in the training set, it is likely that the moderators will never know; the outcome on the moderation algorithm, though, could be profound.

In [40]:

additional_training_data = [
    ('y', 'Deshaun Watson Admits Encounters with Masseuses, Always Consensual'),
    ('y', 'Bruce Jenner is actually getting the Arthur Ashe Courage Award?'),
    ('y', 'The real Tim Tebow: anti gay, anti choice, and a very unexceptional QB who owes a great deal to a teammate who is very good at kicking long fieldgoals when Tim cant get near the red zone.'),
    ('y', 'TIL All NFL players have to do their physicals completely nude and are often nude for over an hour and a half. Many players have fears they were videotaped.'),
    ('y', 'Broncos Brandon Marshall kneels during national anthem, follows Colin Kaepernick’s path'),
    ('y', 'Megan Rapinoe says ‘not many, if any’ US womens soccer player’s would attend White House'),
    ('y', 'With the upcoming Thursday night NFL game, remember that this presents a simplified view of an entire culture, caricatures facial features based on race, depicts an outdated/inaccurate style of headdress, paints them as warmongering aggressors and overly glamorizes the violent side of their history.')
]

utilities.validate_labels(additional_training_data)
training_data.extend(additional_training_data)
display.display_labelled_data(additional_training_data)

Exception: The data labels must be either 'y' or 'n'!

### Calculate probabilities

As described in the lab writeup, the probabilities you need to calculate and subsequently compare for each post are ${P}(Type=On|Post)$ and ${P}(Type=Off|Post)$. We've provided some helper functions to calculate the various components of each probability's equation. 

For each post in the test set, you'll need to calculate these probabilities, compare them, and assign a label based on which probability is higher. 


In [51]:
# Some useful statistics about the data.
#
training_data_statistics = data_tools.DataStats(training_data)

print(utilities.prior_probability('y', training_data_statistics))

0.5008347245409015


We currently have the prior probabilities of both the on-topic and off-topic posts. In order to properly label posts on Sport-It as on-topic or off-topic, we also need to calculate the ${P}(Word|Type=On)$.

In words, this is __the probability of each word appearing in an on-topic of off-topics post__. For this, we provide the `probability_word_given_label` function, demonstrated below.

In [82]:
print(utilities.probability_word_given_label("football", 'y', training_data_statistics))
print(utilities.probability_word_given_label("trump", 'n', training_data_statistics))

0.004848093083387201
0.03533254156769596


## Predicting Labels for Posts
Time to see if our algorithm and model work! We now have the prior probabilities, as well as all the feature probabilities. We can correctly predict what label a post should have. For example if we were given the following post:
<blockquote>The game last night was absolutely terrible!</blockquote>

We should be able to __determine this post's label by the likelihood of it being on-topic and off-topic__:

Once you have the probability of the post being on-topic or off-topic given the words in the posts, __choose the label corresponding to the highest probability of the two to correctly classify that post__. 

Write a function `probability_label_given_post` that returns the probability of a label given a post. Then, using this function, write another function `classify_posts` that takes in the test data and assigns a label to each post.

In [86]:
def probability_label_given_post(post):
    # TODO write your code here
    pass
    
def classify_posts(posts):
    # TODO write your code here
    pass

The code below returns an array of tuples(labels, titles) based on the probabilities we found before. __With your group, quickly discuss the code and what it does__.

In [ ]:
from lab.data_tools import parse_unlabeled_reddit_feed, parse_unlabeled_espn
espn_data = parse_unlabeled_espn('./data/test/espn.txt', limit=100)
politics_data = parse_unlabeled_reddit_feed('./data/test/politics.txt', limit=100)

testing_data = espn_data + politics_data
solution = [('y', e) for e in espn_data] + [('n', p) for p in politics_data]

def filter_posts(posts):
    """
    Input: array of posts to filter WITHOUT labels (see output of parse_unlabeled_espn/reddit_feed)
    Output: array of posts as tuples (label, post title), see output of parse_data
    """
    # your code here!
    result = []
    for submission in testing_data:
        validity = post_validity(submission)
        result.append((validity, submission))

    return result
filtering_result = filter_posts(testing_data)
display.display_labelled_data(filtering_result)

Great! Now we predicted labels for the posts in the testing set, based on the probabilities we calculated. Let’s go ahead and test how accurate your algorithm is! Run the code below to get a percentage of labels you correctly predicted.

In [ ]:
def verify_algorithm(test_result, solution):
    """
    Input: result of the test labelling, and the solution labelling. They should both be arrays of tuples (see output of parse_data for info)
    Output: Entries in test_result that did not appear in solution -- also known as wrong entries
    """
    return list(set(test_result) - set(solution))

In [ ]:
mislabelled = verify_algorithm(filtering_result, solution)
score = (1 - (len(mislabelled) / len(solution)))
print('Your accuracy is:', 100*score,'%')
display.display_labelled_data(mislabelled)

Look at the percentage you got when you ran the code block above. Were you expecting this score? Modify your past code, and see how the percentage changes. Try to get the best possible percentage. __Remember, incorrect moderation has real consequences for Sport-It.__

***

# Part 2: The Consequences of Naivety

### __More about the effects of a Naive content moderation algorithm can be found on the [ethical moderation website](https://dylanirlbeck.github.io/ethical-moderation/project#part-2-the-consequences-of-naivety)__

***

# Part 3: Introducing Human Content Moderation

### __More about human content moderation can be found on the [ethical moderation website](https://dylanirlbeck.github.io/ethical-moderation/project#part-3-introducing-human-content-moderation)__

***

# Resources

 - [At the End of the Day Facebook Does What it Wants](https://s3.amazonaws.com/kvaccaro.com/documents/vaccaro_cscw2020.pdf)
 - [Introduction to Bag-Of-Words Models](https://machinelearningmastery.com/gentle-introduction-bag-words-model/)
 - [Introduction to NBC](https://towardsdatascience.com/introduction-to-naive-bayes-classification-4cffabb1ae54)
 - [Laplace Smoothing](https://towardsdatascience.com/laplace-smoothing-in-na%C3%AFve-bayes-algorithm-9c237a8bdece#:~:text=Laplace%20smoothing%20is%20a%20smoothing%20technique%20that%20helps%20tackle%20the,the%20positive%20and%20negative%20reviews.)
 - [Sample Data Examples](https://www.reddit.com/r/sports/controversial)
